In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/TRAINING

In [ ]:
# %%capture
# !pip install sentence_transformers
# !pip install pypdf
# !pip install xlsxwriter

In [1]:
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
from IPython.core.display import display, HTML

import os

import random
random.seed(42)
import pandas as pd
from helpers import *
import pickle
import numpy as np
import torch
from pypdf import PdfReader
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 
from collections import Counter
import math
from nltk import word_tokenize
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline
import nltk
nltk.download('punkt')

/tmp/ipykernel_1144492/1460843178.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
[nltk_data] Downloading package punkt to /home/roj14702/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Functions

In [2]:
def load_cvs():
    cvs = []
    for i in range(1,6):
        cv = ""
        reader = PdfReader(f"../00_data/CVs/CV_{i}.pdf")

        pages = reader.pages
        for i in range(len(pages)):
            page = reader.pages[i].extract_text().strip()
            cv +=page
        cvs.append(cv)
    return cvs

In [3]:
def load_pickle(filepath):
  with open(filepath, "rb") as fIn:
      stored_data = pickle.load(fIn)
  return stored_data

In [3]:
def setup_classifier():
    path = "../00_data/Classifier/model_classification_jobgbert/"
    model = AutoModelForSequenceClassification.from_pretrained(path)
    tokenizer = AutoTokenizer.from_pretrained(path)
    Classifier = pipeline("text-classification",model,tokenizer=tokenizer)
    return Classifier


In [4]:
def ismanager(ad):
  return "leiter" in ad["jobtitle"].lower() or "führungskraft" in ad["jobtitle"].lower() or "arzt" in ad["jobtitle"].lower() #or ad["esco_id"][0]=="1"

In [5]:
def text_alphanum(text):
  return "".join(x for x in text if x.isalnum() or x.isspace())

In [6]:
def shorten_text(text,pipe):
    annots_jobad = []
    splitted_text = [x for x in text.split("\n") if x != "" or x != " ,"]
    print(len(splitted_text))
    if len(splitted_text) <= 2 or len(splitted_text)>50:
      tokenized = word_tokenize(text, language="german")
      no_chunks = math.ceil(len(tokenized)/20)
      print(no_chunks)
      splitted_text = np.array_split((tokenized), no_chunks)
      splitted_text = [" ".join(x) for x in splitted_text]
    #print(len(splitted_text))
    for paragraph in splitted_text:
        try:
            res = pipe(paragraph)[0]["label"]
        except:
            res = pipe(paragraph[:250])[0]["label"]
        annots_jobad.append({"text":paragraph,"label":res})
        text_short = " ".join([x["text"] for x in annots_jobad if x["label"] == "LABEL_1"])
    return text_short

# Load Data

In [7]:
prefix_path = "../00_data/SBERT_Models/models/"

In [8]:
testads = pd.DataFrame(load_json(r"../00_data/EURES/eures_testads_final_short.json"))

# Load Model

In [9]:
for model in os.listdir(prefix_path):
  print(f"Available model: {model}")

Available model: gbert_batch32_woTSDAE_2e-05_f10
Available model: gbert_batch16_wTSDAE_2e-05_f10_best
Available model: jobgbert_TSDAE_epochs5
Available model: jobgbert_batch32_woTSDAE_2e-05_f10
Available model: gbert_TSDAE_epochs5
Available model: gbert_batch16_wTSDAE_2e-05_f10


In [10]:
paths = [#"deepset/gbert-base",
          #"agne/jobGBERT",
          #"jobgbert_TSDAE_epochs5/",
          #"gbert_TSDAE_epochs5/",
          #"jobgbert_batch16_woTSDAE_2e-05_f10/",
          #"jobgbert_batch16_wTSDAE_2e-05_f10/",
          "jobgbert_batch32_woTSDAE_2e-05_f10/",
          #"jobgbert_batch32_wTSDAE_2e-05_f10/",
          #"jobgbert_batch64_woTSDAE_2e-05_f10/",
          #"jobgbert_batch64_wTSDAE_2e-05_f10/",
          #"gbert_batch16_woTSDAE_2e-05_f10/",
          #"gbert_batch16_wTSDAE_2e-05_f10/",
          "gbert_batch32_woTSDAE_2e-05_f10/",
         # "gbert_batch32_wTSDAE_2e-05_f10/",
          #"gbert_batch64_woTSDAE_2e-05_f10/",
          #"gbert_batch64_wTSDAE_2e-05_f10/",
          ]
          

# Evaluate with Test Ads

In [18]:
MRR = []
MRR_AT = 100
currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])

for model_path in paths:
  print(f"Loading Model {model_path}")
  if model_path in ["agne/jobGBERT","deepset/gbert-base"]:
    model = SentenceTransformer(model_path)
    embeddings = encode_jobs(model)
  else:
    model = SentenceTransformer(f"../00_data/SBERT_Models/models/{model_path}")
    embeddings = load_pickle(f"../00_data/SBERT_Models/models/{model_path}embeddings.pkl")
  print(embeddings.keys())
  print("Creating Embeddings.")
  encodings_short = model.encode(list(testads["short_texts"]), show_progress_bar=True)
 # encodings_long = model.encode(list(testads["description"]), show_progress_bar=True) 

  testads["embeddings_short"] = encodings_short.tolist()
  #testads["embeddings_long"] = encodings_long.tolist()

  print("Finished creating Embeddings. Evaluating.")

  for textkind in ["embeddings_short"]:#,"embeddings_long"]:
    similarities = {}
    for k in ["job_centroid"]:
      similarities[k] = (util.cos_sim(testads[textkind],embeddings[k]["embeddings"]))
    
    for k in similarities:
      ranks = []
      missing = 0
      max_similarity = (max(map(max, similarities[k])))
      simdf = pd.DataFrame(similarities[k],columns=embeddings[k]["esco_id"], index=testads["esco_id"])
      for i in tqdm(range(len(simdf))):
        id = simdf.iloc[i].name
        series = simdf.iloc[i].sort_values(ascending=False).reset_index()
        #print(series)
        rank = (series[series["index"]==id].index.item()+1)
        #print(rank)
        if rank > MRR_AT:
          missing +=1
          ranks.append(0)
        else:
          ranks.append(1/rank)
      missing = missing/len(simdf)
      MRR.append({"model":model_path.split("/")[-2],"textkind": textkind,"embedding_kind":k,
                  "MRR":np.mean(ranks), "missing":missing, "max_similarity": max_similarity, "MRR@":MRR_AT})
      df = pd.DataFrame(MRR).sort_values(by=["MRR"], ascending=[False]).reset_index(drop=True)
      display(df)
      df.to_excel(f"../00_data/SBERT_Models/evaluation/{currently}_evaluation.xlsx")

Loading Model jobgbert_batch32_woTSDAE_2e-05_f10/
dict_keys(['skillsets', 'desc', 'jobtitle', 'adcentroid_filtered', 'adcentroid_unfiltered', 'job_centroid'])
Creating Embeddings.


Batches:   0%|          | 0/71 [00:00<?, ?it/s]

Finished creating Embeddings. Evaluating.


/home/roj14702/miniconda3/envs/careerbert/lib/python3.10/site-packages/sentence_transformers/util.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  a = torch.tensor(a)
100%|███████████████████████████████████████████████████████| 2250/2250 [00:01<00:00, 1351.03it/s]


,model,textkind,embedding_kind,MRR,missing,max_similarity,MRR@
0,jobgbert_batch32_woTSDAE_2e-05_f10,embeddings_short,job_centroid,0.433598,0.105778,tensor(0.9398),100


Loading Model gbert_batch32_woTSDAE_2e-05_f10/
dict_keys(['skillsets', 'desc', 'adcentroid_filtered', 'adcentroid_unfiltered', 'job_centroid'])
Creating Embeddings.


Batches:   0%|          | 0/71 [00:00<?, ?it/s]

Finished creating Embeddings. Evaluating.


100%|███████████████████████████████████████████████████████| 2250/2250 [00:01<00:00, 1261.36it/s]


,model,textkind,embedding_kind,MRR,missing,max_similarity,MRR@
0,jobgbert_batch32_woTSDAE_2e-05_f10,embeddings_short,job_centroid,0.433598,0.105778,tensor(0.9398),100
1,gbert_batch32_woTSDAE_2e-05_f10,embeddings_short,job_centroid,0.414569,0.123111,tensor(0.9497),100


In [ ]:
# df = pd.read_excel(f"/content/drive/MyDrive/TRAINING/_COSINESIM/20230403173019_evaluation.xlsx")

## Test with LLMs

In [12]:
print(testads['description'])
# print(testads)

0       Stellenangebotsbeschreibung: Willkommen im Tea...
1       Stellenangebotsbeschreibung: Umweltingenieurin...
2       Stellenangebotsbeschreibung: Im Umwelt- und Na...
3       Stellenangebotsbeschreibung: Sie suchen einen ...
4       Stellenangebotsbeschreibung: - Proaktive Betre...
                              ...                        
2245    Stellenangebotsbeschreibung: Inhouse Consultan...
2246    Stellenangebotsbeschreibung: Bei einem unserer...
2247    Stellenangebotsbeschreibung: Menschen und Tech...
2248    Stellenangebotsbeschreibung: Die Wasserstraßen...
2249    Stellenangebotsbeschreibung: Für den Fachberei...
Name: description, Length: 2250, dtype: object


In [13]:
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Tuple
import time

In [16]:
load_dotenv()

# Config
client = OpenAI()
MODEL = "gpt-4o-mini"
MRR_AT = 20

In [17]:
def load_data():
    """Load EURES test ads and ESCO jobs"""
    # Load test ads
    testads = pd.DataFrame(load_json("../00_data/EURES/eures_testads_final_short.json"))
    
    # Load ESCO jobs (adjust path as needed)
    with open("../00_data/ESCO/ESCO_JOBS_ALL.json", 'r') as f:
        esco_jobs = json.load(f)
    
    return testads, esco_jobs

In [18]:
def generate_matching_prompt(description: str, esco_jobs: List[Dict]) -> str:
    """Generate and return the prompt for job matching"""
    prompt = f"""Given this job description, identify EXACTLY 20 most likely matching ESCO occupation IDs.
    Consider the job responsibilities, required skills, and overall role.

    Job Description:
    {description}

    Available ESCO Jobs:
    {json.dumps([{
        'id': job['jobid_esco'],
        'title': job['jobtitle'],
        'description': job['jobdescription']
    } for job in esco_jobs], ensure_ascii=False)}

    IMPORTANT: You must return EXACTLY 20 matches, ranked by relevance.
    Return your response in this JSON format:
    {{
        "matches": [
            {{
                "esco_id": "id",
                "confidence": score_0_to_100,
                "reasoning": "brief explanation"
            }}
        ]
    }}
    
    List matches in descending order of confidence, with exactly 20 matches."""
    
    return prompt

In [6]:
def preview_prompt(description: str, esco_jobs: List[Dict]):
    """Print the full prompt that will be sent to the API"""
    prompt = generate_matching_prompt(description, esco_jobs)
    print("=== System Message ===")
    print("You are a job matching expert. Always return exactly 20 matches.\n")
    print("=== User Message ===")
    print(prompt)

In [7]:
def match_job(description: str, esco_jobs: List[Dict], verbose: bool = False) -> List[Dict]:
    """Match a single job description to ESCO jobs"""
    prompt = generate_matching_prompt(description, esco_jobs)
    
    if verbose:
        print("\n=== Using Prompt ===")
        preview_prompt(description, esco_jobs)
        print("\n=== Sending Request to API ===")
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are a job matching expert. Always return exactly 20 matches."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        
        if verbose:
            print("\n=== API Response ===")
            print(response.choices[0].message.content)
        
        result = json.loads(response.choices[0].message.content)
        matches = result['matches']
        
        # Verify we got exactly 20 matches
        if len(matches) != 20:
            print(f"Warning: Got {len(matches)} matches instead of 20")
            # Pad with dummy matches if necessary
            while len(matches) < 20:
                matches.append({
                    "esco_id": "0",
                    "confidence": 0,
                    "reasoning": "Padding match"
                })
            # Trim if we somehow got more
            matches = matches[:20]
            
        return matches
        
    except Exception as e:
        print(f"Error in matching: {e}")
        return []

In [8]:
def test_prompt_generation():
    """Test the prompt generation with a sample job"""
    print("Testing prompt generation with first test job...")
    
    # Load test data
    testads, esco_jobs = load_data()
    
    # Get first job description
    first_job = testads.iloc[0]
    
    # Preview the prompt
    print("\nPrompt Preview for First Job:")
    preview_prompt(first_job['description'], esco_jobs)
    
    return first_job, esco_jobs

In [2]:
# first_job, esco_jobs = test_prompt_generation()

In [9]:
def test_single_match():
    """Test a single job match with full output"""
    first_job, esco_jobs = test_prompt_generation()
    
    print("\nTesting job matching with verbose output...")
    matches = match_job(first_job['description'], esco_jobs, verbose=True)
    
    print("\nMatching Results:")
    for i, match in enumerate(matches, 1):
        print(f"\n{i}. ESCO ID: {match['esco_id']}")
        print(f"   Confidence: {match['confidence']}")
        print(f"   Reasoning: {match['reasoning']}")

In [10]:
# test_single_match()

In [11]:
def evaluate_match(matches: List[Dict], true_esco_id: str) -> Dict:
    """Evaluate a single job match"""
    # Find rank of correct ESCO ID
    correct_rank = None
    for rank, match in enumerate(matches, 1):
        if match['esco_id'] == true_esco_id:
            correct_rank = rank
            break
    
    # Calculate metrics
    mrr = 1/correct_rank if correct_rank and correct_rank <= MRR_AT else 0
    missing = 1 if not correct_rank or correct_rank > MRR_AT else 0
    
    return {
        'correct_rank': correct_rank,
        'mrr': mrr,
        'missing': missing,
        'top_1_hit': correct_rank == 1,
        'top_5_hit': correct_rank is not None and correct_rank <= 5,
        'top_20_hit': correct_rank is not None and correct_rank <= 20,
    }

In [15]:
def process_jobs(testads: pd.DataFrame, esco_jobs: List[Dict], batch_size: int = 5):
    """Process all jobs and evaluate results"""
    MRR = []
    currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])
    
    for i in tqdm(range(0, len(testads), batch_size)):
        batch = testads.iloc[i:i+batch_size]
        
        for _, job in batch.iterrows():
            # Get matches for this job
            matches = match_job(job['description'], esco_jobs)
            
            if not matches:  # Skip if matching failed
                continue
                
            # Evaluate matches
            evaluation = evaluate_match(matches, job['esco_id'])
            
            MRR.append({
                "model": MODEL,
                "job_id": job.name,
                "esco_id": job['esco_id'],
                "MRR": evaluation['mrr'],
                "missing": evaluation['missing'],
                "rank": evaluation['correct_rank'],
                "MRR@": MRR_AT,
                "top_20_hit": evaluation['top_20_hit']
            })
            
            # Display running results
            df = pd.DataFrame(MRR)
            summary = {
                "Mean MRR": df['MRR'].mean(),
                "Missing Rate": df['missing'].mean(),
                "Top-20 Hit Rate": df['top_20_hit'].mean(),
                "Processed Jobs": len(df)
            }
            display(pd.DataFrame([summary]))
            
            # Save results
            df.to_excel(f"../00_data/LLM_evaluation/{currently}_evaluation.xlsx")
        
        # Respect API rate limits
        time.sleep(1)
    
    return df

In [13]:
# Load data
testads, esco_jobs = load_data()

# Process a small batch first to test
test_batch = testads.head(3)
print("Running initial test with 3 jobs...")
test_results = process_jobs(test_batch, esco_jobs)

# If test successful, process all jobs
if input("Continue with full evaluation? (y/n): ").lower() == 'y':
    results_df = process_jobs(testads, esco_jobs)

NameError: name 'pd' is not defined

In [26]:
def create_batch_prompt(description: str, batch: List[Dict], batch_num: int, total_batches: int) -> str:
    """Create prompt for batch processing with stricter matching criteria"""
    return """Analysieren Sie diese Stellenbeschreibung und vergleichen Sie sie mit den ESCO-Berufen.
    Finden Sie nur EXAKTE Übereinstimmungen, keine verwandten oder ähnlichen Berufe.

    Stellenbeschreibung:
    {}

    ESCO-Berufe Batch {}/{}:
    {}

    Bewertungsrichtlinien:
    - Bewerten Sie die Ähnlichkeit streng mathematisch (wie ein Cosinus-Similarity-Score)
    - 90-100: Fast identische Beschreibung
    - 70-89: Sehr hohe Überlappung der Kernaufgaben
    - 50-69: Mittlere Überlappung
    - 30-49: Geringe Überlappung
    - 0-29: Minimale oder keine Überlappung

    Geben Sie nur Matches zurück, die eine echte inhaltliche Übereinstimmung aufweisen.
    Format (JSON):
    {{
        "matches": [
            {{
                "esco_id": "id",
                "confidence": similarity_score,
                "reasoning": "kurze_begruendung_der_uebereinstimmung"
            }}
        ]
    }}""".format(
        description,
        batch_num,
        total_batches,
        json.dumps([{
            'id': job['jobid_esco'],
            'title': job['jobtitle'],
            'description': job['jobdescription'][:200],
            'skills': job['skills'][:5]  # Include top 5 skills for better matching
        } for job in batch], ensure_ascii=False)
    )

In [27]:
def create_ranking_prompt(description: str, matches: List[Dict]) -> str:
    """Create prompt for final ranking with strict comparison criteria"""
    return """Erstellen Sie ein finales Ranking der relevantesten Jobmatches.
    
    WICHTIG: Ranken Sie die Jobs streng nach textueller Ähnlichkeit, wie bei einem Embedding-Vergleich.
    Verwenden Sie die bereits berechneten Confidence-Scores als Basis und verfeinern Sie diese.

    Stellenbeschreibung:
    {}

    Potentielle Matches ({} verfügbar):
    {}

    Kriterien für das Ranking:
    1. Textuelle Ähnlichkeit der Jobbeschreibungen
    2. Übereinstimmung der Kernaufgaben
    3. Übereinstimmung der erforderlichen Fähigkeiten

    Format (JSON):
    {{
        "matches": [
            {{
                "esco_id": "id",
                "confidence": similarity_score,
                "reasoning": "kurze_begruendung"
            }}
        ]
    }}
    
    Geben Sie alle verfügbaren Matches zurück, streng nach Ähnlichkeit sortiert.""".format(
        description,
        len(matches),
        json.dumps(matches, ensure_ascii=False)
    )

In [29]:
def process_matches(matches: List[Dict], expected_count: int = 100) -> List[Dict]:
    """Process matches to ensure consistent scoring and ranking"""
    # Sort by confidence
    sorted_matches = sorted(matches, key=lambda x: float(x['confidence']), reverse=True)
    
    # Normalize confidence scores to better spread across range
    if sorted_matches:
        max_conf = float(sorted_matches[0]['confidence'])
        min_conf = float(sorted_matches[-1]['confidence'])
        conf_range = max_conf - min_conf if max_conf != min_conf else 1
        
        for match in sorted_matches:
            # Normalize and rescale confidence to [0, 100]
            orig_conf = float(match['confidence'])
            norm_conf = (orig_conf - min_conf) / conf_range * 100
            match['confidence'] = round(norm_conf, 2)
    
    return sorted_matches[:expected_count]

In [30]:
def match_job_batched(description: str, esco_jobs: List[Dict], batch_size: int = 50, verbose: bool = False) -> List[Dict]:
    """Match jobs by processing ESCO jobs in batches and combining results"""
    all_matches = []
    total_batches = (len(esco_jobs) + batch_size - 1) // batch_size
    
    if verbose:
        print(f"Processing {total_batches} batches of ESCO jobs...")
    
    # Process ESCO jobs in batches
    for i in range(0, len(esco_jobs), batch_size):
        batch = esco_jobs[i:i+batch_size]
        batch_num = i//batch_size + 1
        
        prompt = create_batch_prompt(description, batch, batch_num, total_batches)

        if verbose:
            print(f"\nVerarbeite Batch {batch_num}/{total_batches}")
        
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Sie sind ein Experte für Jobmatching und analysieren Stellenbeschreibungen in deutscher Sprache. Antworten Sie ausschließlich mit JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            batch_matches = json.loads(response.choices[0].message.content)['matches']
            if verbose:
                print(f"Gefunden: {len(batch_matches)} Matches in diesem Batch")
            all_matches.extend(batch_matches)
            
        except Exception as e:
            print(f"Fehler bei Batch {batch_num}: {e}")
            continue
    
    # Final ranking with improved processing
    if len(all_matches) > 0:
        if verbose:
            print(f"\nErstelle finales Ranking aus {len(all_matches)} potentiellen Matches...")
        
        # First process and normalize all matches
        processed_matches = process_matches(all_matches)
        
        ranking_prompt = create_ranking_prompt(description, processed_matches)

        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Sie sind ein Experte für semantische Textvergleiche und Jobmatching."},
                    {"role": "user", "content": ranking_prompt}
                ],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            final_matches = json.loads(response.choices[0].message.content)['matches']
            
            # Process final matches again to ensure consistency
            final_matches = process_matches(final_matches, min(len(all_matches), 100))
            
            if verbose:
                print(f"Finales Ranking enthält {len(final_matches)} Matches")
            
            return final_matches
            
        except Exception as e:
            print(f"Fehler beim finalen Ranking: {e}")
            # Fallback to processed matches
            return processed_matches
    
    return []

In [31]:
def evaluate_match(matches: List[Dict], true_esco_id: str) -> Dict:
    """Evaluate a single job match"""
    # Find rank of correct ESCO ID
    correct_rank = None
    for rank, match in enumerate(matches, 1):
        if match['esco_id'] == true_esco_id:
            correct_rank = rank
            break
    
    # Calculate metrics
    mrr = 1/correct_rank if correct_rank and correct_rank <= MRR_AT else 0
    missing = 1 if not correct_rank or correct_rank > MRR_AT else 0
    
    return {
        'correct_rank': correct_rank,
        'mrr': mrr,
        'missing': missing,
        'top_1_hit': correct_rank == 1,
        'top_5_hit': correct_rank is not None and correct_rank <= 5,
        'top_20_hit': correct_rank is not None and correct_rank <= 20,
        'top_100_hit': correct_rank is not None and correct_rank <= 100
    }

In [32]:
def process_jobs(testads: pd.DataFrame, esco_jobs: List[Dict], batch_size: int = 50):
    """Process all jobs using batched matching"""
    MRR = []
    currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])
    
    for idx in tqdm(range(len(testads))):
        job = testads.iloc[idx]
        
        # Get matches using batched approach
        matches = match_job_batched(
            job['description'], 
            esco_jobs, 
            batch_size=batch_size,
            verbose=(idx == 0)  # Show verbose output for first job only
        )
        
        if not matches:
            continue
            
        evaluation = evaluate_match(matches, job['esco_id'])
        
        MRR.append({
            "model": MODEL,
            "job_id": job.name,
            "esco_id": job['esco_id'],
            "MRR": evaluation['mrr'],
            "missing": evaluation['missing'],
            "rank": evaluation['correct_rank'],
            "MRR@": MRR_AT,
            "top_20_hit": evaluation['top_20_hit'],
            "top_100_hit": evaluation['top_100_hit']
        })
        
        # Display running results
        df = pd.DataFrame(MRR)
        summary = {
            "Mean MRR": df['MRR'].mean(),
            "Missing Rate": df['missing'].mean(),
            "Top-20 Hit Rate": df['top_20_hit'].mean(),
            "Top-100 Hit Rate": df['top_100_hit'].mean(),
            "Processed Jobs": len(df)
        }
        display(pd.DataFrame([summary]))
        
        df.to_excel(f"../00_data/LLM_evaluation/{currently}_evaluation.xlsx")
        time.sleep(1)
    
    return df

In [37]:
def test_single_job():
    """Test the matching system with a single job"""
    # Load data
    testads, esco_jobs = load_data()
    
    # Get first job
    first_job = testads.iloc[0]
    
    print("Testing job matching with first job...")
    matches = match_job_batched(first_job['description'], esco_jobs, batch_size=50, verbose=True)
    
    print("\nMatching Results:")
    for i, match in enumerate(matches[:10], 1):  # Show top 10 matches
        print(f"\n{i}. ESCO ID: {match['esco_id']}")
        print(f"   Confidence: {match['confidence']}")
        print(f"   Reasoning: {match['reasoning']}")
    
    # Evaluate the match
    evaluation = evaluate_match(matches, first_job['esco_id'])
    print("\nEvaluation:")
    for metric, value in evaluation.items():
        print(f"{metric}: {value}")

In [38]:
test_single_job()

Testing job matching with first job...
Processing 59 batches of ESCO jobs...

Verarbeite Batch 1/59
Gefunden: 1 Matches in diesem Batch

Verarbeite Batch 2/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 3/59
Gefunden: 2 Matches in diesem Batch

Verarbeite Batch 4/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 5/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 6/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 7/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 8/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 9/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 10/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 11/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 12/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 13/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 14/59
Gefunden: 0 Matches in diesem Batch

Verarbeite Batch 15/59
Gefunden: 3 Matches in diesem Batch

Verarbeite Batch 16/59
Gefunden

# Test with TFIDF 

In [ ]:
def calculate_tfidf_similarity(inputtext):
  if isinstance(inputtext,str):
    inputtext = [inputtext]
  path = "../00_data/TF-IDF_Vectorizer"
  vectorizer = load_pickle(path+"1_vectorizer.pkl")
  tfidf_matrix = load_pickle(path+"1_tfidf_matrix.pkl")
  #jobtitles = load_pickle("/content/drive/MyDrive/TRAINING/data/1_jobnames.pkl")
  escoids = load_pickle(path+"1_escoids.pkl")
  inputtfidf = vectorizer.transform(inputtext)
  cosine_sim = cosine_similarity(inputtfidf, tfidf_matrix)
  #cossim_df = pd.DataFrame(cosine_sim, columns = escoids, index=["similarity"]).T.sort_values(by="similarity",ascending=False).reset_index()
  #cossim_df.columns = ["esco_id","similarity"]
  #ranks = list(cossim_df.index+1)
  #cossim_df["rank"] = ranks
  return cosine_sim#,cossim_df

In [ ]:
calculate_tfidf_similarity(testads["short_texts"])

In [ ]:
MRR = []
MRR_AT = 100
currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])

for model_path in paths:
  print(f"Loading Model {model_path}")
  if model_path in ["agne/jobGBERT","deepset/gbert-base"]:
    model = SentenceTransformer(model_path)
    embeddings = encode_jobs(model)
  else:
    model = SentenceTransformer(f"/content/drive/MyDrive/TRAINING/_COSINESIM/content/{model_path}")
    embeddings = load_pickle(f"/content/drive/MyDrive/TRAINING/_COSINESIM/content/{model_path}embeddings.pkl")
  print(embeddings.keys())
  print("Creating Embeddings.")
  encodings_short = model.encode(list(testads["short_texts"]), show_progress_bar=True)
  #encodings_long = model.encode(list(testads["description"]), show_progress_bar=True) 

  testads["embeddings_short"] = encodings_short.tolist()
  #testads["embeddings_long"] = encodings_long.tolist()

  print("Finished creating Embeddings. Evaluating.")

  for textkind in ["embeddings_short"]:
    similarities = {}
    for k in ["job_centroid"]:
        similarities_bert = (util.cos_sim(testads[textkind],embeddings[k]["embeddings"]))
        similarities[k+"_woTFIDF"] = similarities_bert

        max_influence_tfidf = (max(map(max, similarities_bert))).item()*0.2
        similarities_tfidf = calculate_tfidf_similarity(testads["short_texts"])
        similarities_tfidf *= max_influence_tfidf/similarities_tfidf.max()
        
        similarities_combi = np.add(similarities_bert,similarities_tfidf)
        similarities[k+"_wTFIDF"] = similarities_combi
    print(similarities.keys())
    for k in similarities:
      ranks = []
      missing = 0
      simdf = pd.DataFrame(similarities[k],columns=embeddings["job_centroid"]["esco_id"], index=testads["esco_id"])
      for i in tqdm(range(len(simdf))):
        id = simdf.iloc[i].name
        series = simdf.iloc[i].sort_values(ascending=False).reset_index()
        #print(series)
        rank = (series[series["index"]==id].index.item()+1)
        #print(rank)
        if rank > MRR_AT:
          missing +=1
          ranks.append(0)
        else:
          ranks.append(1/rank)
      missing = missing/len(simdf)
      MRR.append({"model":model_path.split("/")[-2],"textkind": textkind,"embedding_kind":k, "MRR":np.mean(ranks), "missing":missing, "MRR@":MRR_AT})
      df = pd.DataFrame(MRR).sort_values(by=["MRR"], ascending=[False]).reset_index(drop=True)
      display(df)
      df.to_excel(f"../00_data/SBERT_Models/Evaluation/{currently}_evaluation.xlsx")